In [1]:
from tensorflow import keras
from tensorflow.keras import layers
from models.vqvae2 import *

In [2]:
model = VQVAETrainer(latent_dim=None, num_embeddings=None)
model.load_weights("./saved_models/savio_model_1")
model.compile(optimizer=keras.optimizers.Adam())

2021-10-30 17:37:47.678305: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
model.vqvae.summary()

Model: "vq_vae"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(None, 1024, 88, 1) 0                                            
__________________________________________________________________________________________________
model (Functional)              (None, 256, 22, 128) 427648      input_5[0][0]                    
__________________________________________________________________________________________________
model_1 (Functional)            (None, 128, 11, 128) 352768      model[0][0]                      
__________________________________________________________________________________________________
conv2d_23 (Conv2D)              (None, 128, 11, 64)  8256        model_1[0][0]                    
_____________________________________________________________________________________________

In [ ]:
def split(model, start, end):
    confs = model.get_config()
    kept_layers = set()
    for i, l in enumerate(confs['layers']):
        if i == 0:
            confs['layers'][0]['config']['batch_input_shape'] = model.layers[start].input_shape
            if i != start:
                confs['layers'][0]['name'] += str(random.randint(0, 100000000)) # rename the input layer to avoid conflicts on merge
                confs['layers'][0]['config']['name'] = confs['layers'][0]['name']
        elif i < start or i > end:
            continue
        kept_layers.add(l['name'])
    # filter layers
    layers = [l for l in confs['layers'] if l['name'] in kept_layers]
    layers[1]['inbound_nodes'][0][0][0] = layers[0]['name']
    # set conf
    confs['layers'] = layers
    confs['input_layers'][0][0] = layers[0]['name']
    confs['output_layers'][0][0] = layers[-1]['name']
    # create new model
    submodel = Model.from_config(confs)
    for l in submodel.layers:
        orig_l = model.get_layer(l.name)
        if orig_l is not None:
            l.set_weights(orig_l.get_weights())
    return submodel


In [6]:
random_input = tf.random.uniform(shape=[16, 1024, 88, 1])

In [8]:
model.top_and_bottom_indices(random_input)

(<tf.Tensor: shape=(22528,), dtype=int64, numpy=array([353, 353, 353, ..., 353, 353, 447])>,
 <tf.Tensor: shape=(22528,), dtype=int64, numpy=array([384, 120, 369, ..., 120, 369, 369])>)